<h1 align=right><span style='font-family:Arial Black'>Henry</span></h1>
<p><img src="formas/ETL.jpg", width="300"></p>

# ETL: *`Extracción, transformación y carga de datos`*

## *Para iniciar nuestro trabajo, ocuparemos los*:

### 1. *`Archivos Originales gz`*:
* `steam_games.json.gz`
* `user_reviews.json.gz`
* `users_items.json.gz`

### 2. *`De los anteriores se obtuvieron los respectivos json en folder zip`*:
* `steam_games.json`
* `user_reviews.json`
* `users_items.json`

### 3. *`De los archivos zip se extraen los correspondientes json`*:
* `output_steam_games.json`
* `australian_user_reviews.json`
* `australian_user_items.json`

### Obtenidos los archivos de trabajo, borré sus correspondientes folders para alojarlos en la carpeta `data`.
* Mismos que quedaron en `Desktop` y a un lado del proyecto `PI_ML_OPS`.
* Estos archivos no podrán ser vistos en `GitHub`, cuando suba el proyecto.
* Los archivos aparecen junto con el `Diccionario de Datos STEAM`.
* Capturamos la ubicación del folder `data` y su apertura para ver contenido.

<p><img src="formas/data.png", width="500"></p>
<p><img src="formas/data_open.png", width="700"></p>

In [1]:
# Para iniciar el ETL, necesitamos importar las librerias necesarias para trabajar en la apertura de archivos json.
import pandas as pd
import ast
import warnings
warnings.filterwarnings('ignore')

### Empezando por abrir el archivo:*`steam_games.json`*
* Este archivo no lo pude abrir sin `lines=True`.
* Aunque este parametro no es nativo de Python.
* Ello, puede abrir un archivo `JSON en Python` y 
* puede procesar cada `línea del archivo JSON`, 
* si el `archivo JSON está formateado` de tal manera
* que cada línea sea un `objeto JSON independiente`. 
* Esto es comúnmente conocido como `JSON Line`

In [2]:
df_games = pd.read_json('../data/output_steam_games.json',lines=True)

In [3]:
df_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [4]:
# Eliminamos los valores nulos y revisamos el tipo de columnas, y vemos que
# Las columnas: 'genres','tags' y 'specs' aparecen como lista cada uno de sus registros.
df_games = df_games.dropna()
df_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL


In [5]:
# Revisando las columnas del archivo abierto
df_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [6]:
# Checamos cantidad de registros y columnas 
df_games.shape

(22530, 13)

In [7]:
# Verificamos columna 'release_date' para extaer la columna 'year'
df_games.release_date.sort_values()

89687     1983-06-19
89855     1984-04-29
110028    1984-11-01
116377    1985-01-01
99223     1986-05-01
             ...    
109456      Oct 2016
101344          SOON
120318         SOON™
119928      Sep 2009
90917       Sep 2014
Name: release_date, Length: 22530, dtype: object

In [8]:
# Eliminamos los datos: 'SOON' y ' SOON™', de la columna:'release_date'
df_games= df_games.drop([101344,120318])

In [9]:
# Verificamos la eliminación de los indices anteriores
df_games.release_date.sort_values()

89687     1983-06-19
89855     1984-04-29
110028    1984-11-01
116377    1985-01-01
99223     1986-05-01
             ...    
88819       Oct 2010
88816       Oct 2010
109456      Oct 2016
119928      Sep 2009
90917       Sep 2014
Name: release_date, Length: 22528, dtype: object

In [10]:
# Extraemos la columna 'release_year' de la columna 'release_date', usando el método 'to_datetime' de la librería 'pandas'
df_games['release_year'] = pd.to_datetime(df_games['release_date'],errors='coerce').apply(lambda x:str(x).split('-')[0] if x != None else None)

In [11]:
# Convertimos los registros de la columna 'release_year' a 'integer'
df_games.release_year = df_games.release_year.astype(int)

In [12]:
# Verificamos el total de columnas y registros. 
# Y de paso el tipo de dato de la columna 'year'
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22528 entries, 88310 to 120443
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22528 non-null  object 
 1   genres        22528 non-null  object 
 2   app_name      22528 non-null  object 
 3   title         22528 non-null  object 
 4   url           22528 non-null  object 
 5   release_date  22528 non-null  object 
 6   tags          22528 non-null  object 
 7   reviews_url   22528 non-null  object 
 8   specs         22528 non-null  object 
 9   price         22528 non-null  object 
 10  early_access  22528 non-null  float64
 11  id            22528 non-null  float64
 12  developer     22528 non-null  object 
 13  release_year  22528 non-null  int32  
dtypes: float64(2), int32(1), object(11)
memory usage: 2.5+ MB


In [13]:
# Eliminamos las siguientes columnas porque no las voy a necesitar.
cols = ['publisher','app_name','url','release_date','tags','reviews_url','specs','price','early_access','developer']
df_games = df_games.drop(cols,axis=1)

In [14]:
# Verificamos el resultado, donde vemos que reducimos a 4 columnas y 22528 registros
df_games.head(2)

,genres,title,id,release_year
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,761140.0,2018
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,643980.0,2018


In [15]:
# Cambiamos el objeto lista, apilando los datos en la columna 'genres' y salvando en la variable(columna) 'gen'
gen = df_games.apply(lambda x: pd.Series(x['genres'],dtype='object'),axis=1).stack().reset_index(level=1,drop=True)
# Renombramos columna 'gen' como 'genero'
gen.name = 'genero'

In [16]:
gen

88310         Action
88310         Casual
88310          Indie
88310     Simulation
88310       Strategy
             ...    
120442         Indie
120442        Racing
120442    Simulation
120443        Casual
120443         Indie
Name: genero, Length: 55607, dtype: object

In [17]:
# Eliminamos la columna:`genres` y usando el método 'join' pegamos la columna 'gen', renombrada como 'genero'
df_games = df_games.drop('genres',axis=1).join(gen)

In [18]:
# Vericamos el resultado
df_games

,title,id,release_year,genero
88310,Lost Summoner Kitty,761140.0,2018,Action
88310,Lost Summoner Kitty,761140.0,2018,Casual
88310,Lost Summoner Kitty,761140.0,2018,Indie
88310,Lost Summoner Kitty,761140.0,2018,Simulation
88310,Lost Summoner Kitty,761140.0,2018,Strategy
...,...,...,...,...
120442,Russian Roads,610660.0,2018,Indie
120442,Russian Roads,610660.0,2018,Racing
120442,Russian Roads,610660.0,2018,Simulation
120443,EXIT 2 - Directions,658870.0,2017,Casual


In [19]:
# Verificamos los tipos de datos de los 55607 registros y 4 columnas
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55607 entries, 88310 to 120443
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         55607 non-null  object 
 1   id            55607 non-null  float64
 2   release_year  55607 non-null  int32  
 3   genero        55607 non-null  object 
dtypes: float64(1), int32(1), object(2)
memory usage: 1.9+ MB


In [20]:
# Salvamos el DataFrame como 'df_games.csv' 
#df_games.to_csv('df_games.csv',index=False)

#### Abriendo el archivo: *`australian_user_reviews.json`*

In [21]:
# Una vez extraido el archivo.json: `australian_user_reviews.json`, procedemos a abrirlo con `open` y 
# con el uso de la libreria `ast`. Para esto, partimos de una lista vacia con el metodo `append` vamos 
# completando nuestro archivo `evaluando cada linea` del archivo.json. Mientras el archivo.json, 
# se apoya en el parámetro: encoding='utf-8' para ser leído y abierto correctamente.
lista = []
with open("../data/australian_user_reviews.json",encoding='utf-8') as f:
    for line in f.readlines():
        lista.append(ast.literal_eval(line))
#lista

#### Una vez abierto el archivo `australian_user_reviews.json`:
* `Revisamos si hay columnas a desanidar para tomar los datos`.
* `Se corre nuevamente, sin abrir la lista`.
* `Enseguida, se transforma la lista en DataFrame`

In [22]:
# Se abre el DataFrame y observamos que hay 3 columnas.
# Una de ellas, la columna 'reviews', se encuentra anidada.
df = pd.DataFrame(lista)
df.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


#### Antes de desanidar la columna: `reviews`
* Creamos la columna: `sentiment_analysis`
* Con la librería: `TextBlob`

In [23]:
# Importamos la libreria capaz de ayudarnos en la tarea de analisis de sentimientos.
from textblob import TextBlob 
def sentiment_analysis(review):
    '''Creamos una función de analisis de sentimientos con parametro `review`, usando
       la libreria `TextBlob` para pegarle el metodo `sentiment.polarity` y generar tres
       (`0`:Negative,`1`:Neutral,`2`:Positive) mediciones del sentimiento a evaluar.
       Al final que nos retorne 1 si falta el dato correspondiente.'''
    if isinstance(review,list) and len(review) > 0:
        text = review[0].get('review','')
        sentiment = TextBlob(text).sentiment.polarity
        if sentiment < -0.2:
            return 0 # Negative
        elif sentiment >= -0.2 and sentiment <= 0.2:
            return 1 # Neutral
        else:
            return 2 # Positive
    else:
        return 1

#### Una vez creada la función `sentiment_analysis`
* Aplicamos la función `sentiment_analysis`, a la columna `reviews`

In [24]:
# Creamos la columna:`sentiment_analysis`, aplicando la funcion:`sentiment_analysis` a la columna:`reviews`.
df['sentiment_analysis'] = df['reviews'].apply(sentiment_analysis)
df.head()

,user_id,user_url,reviews,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...",1
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...",1
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',...",2
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2...",2
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',...",1


#### Para desanidar la columna: `reviews`. 
* Vamos a iterar las columnas del archivo: `australian_user_reviews.json`
* Para este propósito, usaremos el método:`iterrows`
* Y mediante un diccionario alojaremos el `archivo`, incluyendo las columnas desanidadas y solo las columnas que queremos conservar.

In [25]:
# Creamos una lista vacia para alojar nuestros datos desanidados
lista_desanidada = []
# `iterrows`, se usa para iterar atravez de las filas del archivo.
for index,row in df.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    sentiment_analysis_value = row['sentiment_analysis']
    reviews = row['reviews']
# Le pegamos a la lista_desanidada, el siguiente diccionario. 
# Incluyendo solo las listas a conservar
    for e in reviews:
        dicc_row = {
            'user_id': user_id,
            'user_url': user_url,
            'reviews': reviews,
            'sentiment_analysis': sentiment_analysis_value,
            'posted': e.get('posted', ''),
            'item_id': e.get('item_id', ''),
            'recommend': e.get('recommend',False),
            'review': e.get('review', '')
        }
        lista_desanidada.append(dicc_row)

#### Transformamos a `DataFrame`:
- Eliminamos la columna `reviews`

In [26]:
df_desanidada = pd.DataFrame(lista_desanidada)

In [27]:
# Antes de abrir el dataset, eliminamos la columna reviews
df_desanidada.drop(['reviews'],axis=1,inplace=True)
df_desanidada.head()

,user_id,user_url,sentiment_analysis,posted,item_id,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,1,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,1,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


#### Convertimos la columna *`posted`* a formato fecha *`AAAA-MM-DD`*
* Usamos las librerías: `dateutil y parser`
* Eliminamos la palabra **`Posted`** de cada registro de la columna *`posted`*
* Aplicamos la función `parse_date`a la columna `posted`

In [28]:
# Importamos la libreria `parser` de `dateutil` para covertir a formato fecha(AAAA-MM-DD).
from dateutil import parser
# Enseguida creamos una funcion para tal efecto y sustituyendo la palabra:
# `Posted` que aparece en cada registro, por ""
def parse_date(date_str):
    try:
        return parser.parse(date_str.replace('Posted',""),fuzzy=True)
    except ValueError:
        return None

In [29]:
# Tomamos la columna `posted` y le aplicamos la función previamente creada.
# Creamos la columna 'posted_date' y eliminamos la columna 'posted'
df_desanidada['posted_date'] = df_desanidada['posted'].apply(parse_date)
df_desanidada = df_desanidada.drop(['posted'],axis=1)

In [30]:
# Abrimos el DataFrame para verificar los registros y columnas
df_desanidada.head()

,user_id,user_url,sentiment_analysis,item_id,recommend,review,posted_date
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,1250,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,22200,True,It's unique and worth a playthrough.,2011-07-15
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,js41637,http://steamcommunity.com/id/js41637,1,251610,True,I know what you think when you see this title ...,2014-06-24
4,js41637,http://steamcommunity.com/id/js41637,1,227300,True,For a simple (it's actually not all that simpl...,2013-09-08


In [31]:
# Verificamos, la eliminación de las columnas anteriores y la reducción de las mismas.
# Contamos ahora con los mismos 59305 registros y 7 columnas.
df_desanidada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   user_id             59305 non-null  object        
 1   user_url            59305 non-null  object        
 2   sentiment_analysis  59305 non-null  int64         
 3   item_id             59305 non-null  object        
 4   recommend           59305 non-null  bool          
 5   review              59305 non-null  object        
 6   posted_date         59280 non-null  datetime64[ns]
dtypes: bool(1), datetime64[ns](1), int64(1), object(4)
memory usage: 2.8+ MB


In [32]:
# Creamos la columna 'posted_year', extrayendola de la columna 'posted_date' a partir del metodo 'to_datetime' de la librería 'pandas'
df_desanidada['posted_year'] = pd.to_datetime(df_desanidada['posted_date'],errors='coerce').apply(lambda x:str(x).split('-')[0] if x != None else None)

In [33]:
# Abrimos el archivo, para ver el resultado
df_desanidada.head(2)

,user_id,user_url,sentiment_analysis,item_id,recommend,review,posted_date,posted_year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,1250,True,Simple yet with great replayability. In my opi...,2011-11-05,2011
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,22200,True,It's unique and worth a playthrough.,2011-07-15,2011


In [34]:
# Verificamos tipo de dato de la columna 'posted_year'
df_desanidada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   user_id             59305 non-null  object        
 1   user_url            59305 non-null  object        
 2   sentiment_analysis  59305 non-null  int64         
 3   item_id             59305 non-null  object        
 4   recommend           59305 non-null  bool          
 5   review              59305 non-null  object        
 6   posted_date         59280 non-null  datetime64[ns]
 7   posted_year         59305 non-null  object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(5)
memory usage: 3.2+ MB


In [35]:
# Convertimos a número los tipos de registro de la columna 'posted_year'
df_desanidada['posted_year'] = pd.to_numeric(df_desanidada['posted_year'],errors='coerce')

In [36]:
# Verificamos los datos de la columna 'posted_year'
df_desanidada['posted_year'].sort_values()

29806    2010.0
15693    2010.0
24664    2010.0
5358     2010.0
45134    2010.0
          ...  
49410       NaN
49411       NaN
52326       NaN
56697       NaN
57240       NaN
Name: posted_year, Length: 59305, dtype: float64

In [37]:
# Encontramos valores nulos en la columna 'posted_year' y los eliminamos
df_desanidada = df_desanidada.dropna(subset=['posted_year'])

In [38]:
# Checamos el tipo de dato de la columna 'posted_year'
df_desanidada.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59280 entries, 0 to 59304
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   user_id             59280 non-null  object        
 1   user_url            59280 non-null  object        
 2   sentiment_analysis  59280 non-null  int64         
 3   item_id             59280 non-null  object        
 4   recommend           59280 non-null  bool          
 5   review              59280 non-null  object        
 6   posted_date         59280 non-null  datetime64[ns]
 7   posted_year         59280 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 3.7+ MB


In [39]:
# Como vemos que el tipo de dato de la columna 'year' es 'float', lo convertimos a 'integer'
df_desanidada['posted_year'] = df_desanidada['posted_year'].astype(int)

In [40]:
# Verificamos la correcta conversión del dato de la columna 'year' a 'integer' 
df_desanidada.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59280 entries, 0 to 59304
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   user_id             59280 non-null  object        
 1   user_url            59280 non-null  object        
 2   sentiment_analysis  59280 non-null  int64         
 3   item_id             59280 non-null  object        
 4   recommend           59280 non-null  bool          
 5   review              59280 non-null  object        
 6   posted_date         59280 non-null  datetime64[ns]
 7   posted_year         59280 non-null  int32         
dtypes: bool(1), datetime64[ns](1), int32(1), int64(1), object(4)
memory usage: 3.4+ MB


In [41]:
# Eliminamos la columna 'posted_date' y verificamos resultado
col = ['user_url','posted_date']
df_desanidada = df_desanidada.drop(col,axis=1)
df_desanidada.head(2)

,user_id,sentiment_analysis,item_id,recommend,review,posted_year
0,76561197970982479,1,1250,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,1,22200,True,It's unique and worth a playthrough.,2011


In [42]:
# Tenemos 59280 registros y 6 columnas
df_desanidada.shape

(59280, 6)

In [43]:
# Salvamos el archivo como 'df_reviews.csv'
#df_desanidada.to_csv('df_reviews.csv',index=False)

### Abriendo el archivo:*`australian_users_items.json`*

In [44]:
# Una vez extraido el archivo.json: `australian_users_items.json`, procedemos a abrirlo con `open` y 
# partiendo de una lista vacia con el metodo `append` vamos completando nuestro archivo `evaluando cada linea` 
# cada line del archivo.json con ayuda de la libreria `ast`. Mientras el archivo.json, se apoya en el parámetro:
# encoding='utf-8' para ser leído y abierto correctamente.
lista = []
with open("../data/australian_users_items.json",encoding='utf-8') as f:
    for line in f.readlines():
        lista.append(ast.literal_eval(line))

#lista

#### Una vez abierto el archivo `australian_users_items.json`:
* `Revisamos si hay columnas a desanidar para tomar los datos`.
* `Se corre nuevamente, sin abrir la lista`.
* `Enseguida, se transforma la lista en DataFrame`

In [45]:
# Se abre el DataFrame y observamos que la columna 'items', se encuentra anidada.
df_items = pd.DataFrame(lista)
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [46]:
# Creamos una lista vacia para alojar nuestros datos desanidados
lista_desanidada = []
#`iterrows`, se usa para iterar atravez de las filas de nuestro archivo
for index,row in df_items.iterrows():
    user_id = row['user_id']
    items_count = row['items_count']
    steam_id = row['steam_id']
    user_url = row['user_url']
# A la lista_desanidada, le anexamos el siguiente diccionario.
# Incluyendo solo las columnas a conservar
    for i in row['items']:
        dicc_row = {
            'user_id': user_id,
            'items_count': items_count,
            'steam_id':steam_id,
            'user_url': user_url,
            'item_id': i['item_id'],
            'item_name': i['item_name'],
            'playtime_forever': i['playtime_forever']
        }
        lista_desanidada.append(dicc_row)

#### Desanidamos la columna: *`items`* para aplicarla al archivo: *`australian_users_items.json`*

#### Una vez hecho el procedimiento anterior, transformamos a DataFrame la `lista_desanidada`

In [47]:
# Abrimos el DataFrame para ver el resultado del desanide de la columna 'items'
df_items_des = pd.DataFrame(lista_desanidada)
df_items_des.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0


In [48]:
# Los registros y columnas de nuestro dataset
df_items_des.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 7 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   steam_id          object
 3   user_url          object
 4   item_id           object
 5   item_name         object
 6   playtime_forever  int64 
dtypes: int64(2), object(5)
memory usage: 275.2+ MB


In [49]:
# Ahora vamos a eliminar otra columna que no ocupamos.
# Quedandonos con 3 columnas de las 6 iniciales
cols = ['items_count','steam_id','user_url','item_name']
df_items_des = df_items_des.drop(cols,axis=1)
df_items_des.head()

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,20,0
2,76561197970982479,30,7
3,76561197970982479,40,0
4,76561197970982479,50,0


In [50]:
# Salvamos el DataFrame como 'df_items.csv'
#df_items_des.to_csv('df_items.csv',index=False)

In [51]:
# Abrimos los datasets de los 3 archivos.
df_games = pd.read_csv('../data/df_games.csv')
df_reviews = pd.read_csv('../data/df_reviews.csv')
df_items = pd.read_csv('../data/df_items.csv')

In [52]:
# Fusionamos los archivos 'reviews' y 'games', mediante el método 'join'
df = df_reviews.join(df_games)
df.head(2)

,user_id,sentiment_analysis,item_id,recommend,review,posted_year,title,id,release_year,genero
0,76561197970982479,1,1250,True,Simple yet with great replayability. In my opi...,2011,Lost Summoner Kitty,761140.0,2018.0,Action
1,76561197970982479,1,22200,True,It's unique and worth a playthrough.,2011,Lost Summoner Kitty,761140.0,2018.0,Casual


In [53]:
# Producto de la fusión anterior, se generan 10 columnas y 59280 registros
df.shape

(59280, 10)

In [54]:
# Fusionamos 'items' con 'df', usando el método 'merge'
df1 = df_items.merge(df,on=['user_id','item_id'])
df1.head(2)

,user_id,item_id,playtime_forever,sentiment_analysis,recommend,review,posted_year,title,id,release_year,genero
0,76561197970982479,22200,271,1,True,It's unique and worth a playthrough.,2011,Lost Summoner Kitty,761140.0,2018.0,Casual
1,76561197970982479,1250,10006,1,True,Simple yet with great replayability. In my opi...,2011,Lost Summoner Kitty,761140.0,2018.0,Action


In [55]:
# Convertimos a número los tipos de registro de la columna 'release_year'
df1['release_year'] = pd.to_numeric(df1['release_year'],errors='coerce')

In [56]:
# Verificamos el resultado
df1['release_year']

0        2018.0
1        2018.0
2        2018.0
3        2018.0
4        2018.0
          ...  
46291       NaN
46292       NaN
46293       NaN
46294       NaN
46295       NaN
Name: release_year, Length: 46296, dtype: float64

In [57]:
# Vemos que hay valores nulos en la columna 'release_year', y los eliminamos
df1 = df1.dropna(subset=['release_year'])

In [58]:
# Vemos que la columna 'release_year' aparece como 'float', entonces la convertimos a 'integer'
df1['release_year'] = df1['release_year'].astype(int)

In [60]:
# Verificamos la conversión y nos damos cuenta
# que tenemos 43881 registros y 11 columnas y 
# la columna 'review' tiene solamente 43863 registros
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43881 entries, 0 to 43880
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             43881 non-null  object 
 1   item_id             43881 non-null  int64  
 2   playtime_forever    43881 non-null  int64  
 3   sentiment_analysis  43881 non-null  int64  
 4   recommend           43881 non-null  bool   
 5   review              43863 non-null  object 
 6   posted_year         43881 non-null  int64  
 7   title               43881 non-null  object 
 8   id                  43881 non-null  float64
 9   release_year        43881 non-null  int32  
 10  genero              43881 non-null  object 
dtypes: bool(1), float64(1), int32(1), int64(4), object(4)
memory usage: 3.6+ MB


In [61]:
# Así que, checamos los valores nulos de la columna 'review'
df1.review.isnull().sum()

18

In [62]:
# Nos damos cuenta que hay 18 valores nulos de la columna 'review'.
# Entonces, los eliminamos
df1 = df1.dropna(subset=['review'])

In [63]:
# Verificamos el resultado de nuestras operaciones y de la información del dataset. 
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43863 entries, 0 to 43880
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             43863 non-null  object 
 1   item_id             43863 non-null  int64  
 2   playtime_forever    43863 non-null  int64  
 3   sentiment_analysis  43863 non-null  int64  
 4   recommend           43863 non-null  bool   
 5   review              43863 non-null  object 
 6   posted_year         43863 non-null  int64  
 7   title               43863 non-null  object 
 8   id                  43863 non-null  float64
 9   release_year        43863 non-null  int32  
 10  genero              43863 non-null  object 
dtypes: bool(1), float64(1), int32(1), int64(4), object(4)
memory usage: 3.6+ MB


In [64]:
# Ahora si, con un dataset de trabajo completamente limpio y compliendo cabalmente con todas las transformaciones, 
# salvamos este archivo como: 'df_trabajo.csv'
#df1.to_csv('df_trabajo.csv',index=False)

### **Salvados los respectivos datasets.csv**:
* Los envie al folder `data` para aligerar el proyecto
* Capturamos la actualización 

<p><img src="formas/data_plus.png", width="700"></p>

<h6 align=right><i>Cohorte</i>:DataPT04</h6>
<h6 align=right><i>Mexico - 2023</i></h6>
<h6 align=left><i>Pag. 1</i></h6>